# Wind Statistics

### Introduction:

The data have been modified to contain some missing values, identified by NaN.  
Using pandas should make this exercise
easier, in particular for the bonus question.

You should be able to perform all of these operations without using
a for loop or other looping construct.


1. The data in 'wind.data' has the following format:

In [1]:
"""
Yr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL
61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04
61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83
61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71
"""

'\nYr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL\n61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04\n61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83\n61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71\n'

   The first three columns are year, month and day.  The
   remaining 12 columns are average windspeeds in knots at 12
   locations in Ireland on that day.   

   More information about the dataset go [here](wind.desc).

### Step 1. Import the necessary libraries

In [2]:
import numpy as np
import pandas as pd

### Step 2. Import the dataset from this [address](https://github.com/guipsamora/pandas_exercises/blob/master/Stats/Wind_Stats/wind.data)

### Step 3. Assign it to a variable called data and replace the first 3 columns by a proper datetime index.

In [3]:
data = pd.read_csv('wind.data', sep='\s+')
data["Period"] = data["Yr"].map(str) + "-" + data["Mo"].map(str) + "-" + data["Dy"].map(str)
data["Period"] = pd.to_datetime(data["Period"])
data = data.loc[:, "RPT":"Period"].set_index("Period")


### Step 4. Year 2061? Do we really have data from this year? Create a function to fix it and apply it.

In [4]:
data.index = data.index.map(lambda x: x - pd.DateOffset(years = 100))

### Step 5. Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns].

In [5]:
data.index.dtype_str

'datetime64[ns]'

### Step 6. Compute how many values are missing for each location over the entire record.  
#### They should be ignored in all calculations below. 

In [6]:
np.count_nonzero(data.isnull())

31

### Step 7. Compute how many non-missing values there are in total.

In [7]:
np.count_nonzero(~data.isnull())

78857

### Step 8. Calculate the mean windspeeds of the windspeeds over all the locations and all the times.
#### A single number for the entire dataset.

In [8]:
data.fillna(value = 0, inplace=True)
np.mean(np.mean(data))

10.223864592840467

### Step 9. Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days 

#### A different set of numbers for each location.

In [64]:
loc_stats = pd.DataFrame([np.min(data), np.max(data), np.mean(data), np.std(data)]).transpose()
loc_stats.columns = ['Min', 'Max', 'Mean', 'Std']
loc_stats.head()


,Min,Max,Mean,Std
BEL,0.13,42.38,13.121,5.83459
BIR,0,26.16,7.09225,3.96838
CLA,0,31.08,8.49247,4.50086
CLO,0,28.21,8.70601,4.50455
DUB,0,30.37,9.79287,4.98044


### Step 10. Create a DataFrame called day_stats and calculate the min, max and mean windspeed and standard deviations of the windspeeds across all the locations at each day.

#### A different set of numbers for each day.

In [10]:
def series_for(fn):
    return data.apply(fn, axis = 1)


data_stats = pd.DataFrame([series_for(np.min), series_for(np.max), series_for(np.mean), series_for(np.std)]).transpose()
data_stats.columns = ['Min', 'Max', 'Mean', 'Std']
data_stats.head()

,Min,Max,Mean,Std
Period,,,,
1961-01-01,0.00,18.50,11.933333,4.418220
1961-01-02,0.00,17.54,10.391667,4.276881
1961-01-03,0.00,18.50,10.671667,4.652977
1961-01-04,1.79,11.75,6.619167,3.061972
1961-01-05,6.17,13.33,10.630000,2.341250


### Step 11. Find the average windspeed in January for each location.  
#### Treat January 1961 and January 1962 both as January.

In [11]:
jan = data.index.map(lambda x: x.month) == 1
jandata = data[jan]
jandata.apply(np.mean, axis = 0)

RPT    14.820717
VAL    12.891416
ROS    13.299624
KIL     7.199498
SHA    11.625914
BIR     8.054839
DUB    11.819355
CLA     9.495000
MUL     9.543208
CLO    10.053566
BEL    14.550520
MAL    18.028763
dtype: float64

### Step 12. Downsample the record to a yearly frequency for each location.

In [14]:
data[data.index.map(lambda x: x.month == 1 and x.day == 1)]

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Period,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,0.00,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1962-01-01,9.29,3.42,11.54,3.50,2.21,1.96,10.41,2.79,3.54,5.17,4.38,7.92
1963-01-01,15.59,13.62,19.79,8.38,12.25,10.00,23.45,15.71,13.59,14.37,17.58,34.13
1964-01-01,25.80,22.13,18.21,13.25,21.29,14.79,14.12,19.58,13.25,16.75,28.96,21.00
1965-01-01,9.54,11.92,9.00,4.38,6.08,5.21,10.25,6.08,5.71,8.63,12.04,17.41
1966-01-01,22.04,21.50,17.08,12.75,22.17,15.59,21.79,18.12,16.66,17.83,28.33,23.79
1867-01-01,6.46,4.46,6.50,3.21,6.67,3.79,11.38,3.83,7.71,9.08,10.67,20.91
1868-01-01,30.04,17.88,16.25,16.25,21.79,12.54,18.16,16.62,18.75,17.62,22.25,27.29
1869-01-01,6.13,1.63,5.41,1.08,2.54,1.00,8.50,2.42,4.58,6.34,9.17,16.71


### Step 13. Downsample the record to a monthly frequency for each location.

In [15]:
data[data.index.map(lambda x: x.day == 1)]

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Period,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,0.00,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1961-02-01,14.25,15.12,9.04,5.88,12.08,7.17,10.17,3.63,6.50,5.50,9.17,8.00
1961-03-01,12.67,13.13,11.79,6.42,9.79,8.54,10.25,13.29,0.00,12.21,20.62,0.00
1961-04-01,8.38,6.34,8.33,6.75,9.33,9.54,11.67,8.21,11.21,6.46,11.96,7.17
1961-05-01,15.87,13.88,15.37,9.79,13.46,10.17,9.96,14.04,9.75,9.92,18.63,11.12
1961-06-01,15.92,9.59,12.04,8.79,11.54,6.04,9.75,8.29,9.33,10.34,10.67,12.12
1961-07-01,7.21,6.83,7.71,4.42,8.46,4.79,6.71,6.00,5.79,7.96,6.96,8.71
1961-08-01,9.59,5.09,5.54,4.63,8.29,5.25,4.21,5.25,5.37,5.41,8.38,9.08
1961-09-01,5.58,1.13,4.96,3.04,4.25,2.25,4.63,2.71,3.67,6.00,4.79,5.41


### Step 14. Downsample the record to a weekly frequency for each location.

In [27]:
data[data.index.dayofweek == data.index[0].dayofweek]

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Period,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,0.00,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1961-01-08,10.96,9.75,7.62,5.91,9.62,7.29,14.29,7.62,9.25,10.46,16.62,16.46
1961-01-15,12.04,9.67,11.75,2.37,7.38,3.13,2.50,6.83,4.75,5.63,7.54,6.75
1961-01-22,9.59,5.88,9.92,2.17,6.87,5.50,9.38,7.04,6.34,7.50,10.88,9.92
1961-01-29,0.00,23.91,22.29,17.54,24.08,19.70,22.00,20.25,21.46,19.95,27.71,23.38
1961-02-05,22.00,16.75,19.70,11.63,14.25,12.04,13.96,15.54,13.37,13.13,19.33,14.67
1961-02-12,16.00,16.92,17.25,8.38,11.75,10.37,11.71,14.37,10.37,14.62,25.41,17.04
1961-02-19,16.17,15.59,13.70,0.00,13.50,10.96,9.62,11.83,11.58,13.67,29.63,20.54
1961-02-26,25.80,21.50,22.42,15.21,23.09,16.88,21.17,17.25,18.46,20.04,22.83,23.87


### Step 15. Calculate the mean windspeed for each month in the dataset.  
#### Treat January 1961 and January 1962 as *different* months.
#### (hint: first find a  way to create an identifier unique for each month.)

In [56]:
data['monthyear'] = data.index.map(lambda x: str(x.month) + "-" + str(x.year))

data.groupby('monthyear').agg(np.mean)

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
monthyear,,,,,,,,,,,,
1-1867,13.736129,11.470968,11.537419,6.928065,10.833548,6.730323,9.787419,7.474839,7.627097,10.169032,13.457742,17.124516
1-1868,14.733226,12.829032,12.294194,7.849032,12.262581,7.133226,12.395806,9.225806,9.415484,9.255161,14.213871,19.095484
1-1869,12.824839,12.113871,11.403871,6.708710,10.557097,5.926452,9.882903,8.375161,8.378065,9.515484,13.410000,16.957742
1-1870,13.152581,11.731935,13.083871,5.810968,9.507742,8.040323,9.843226,8.834194,8.407097,7.626452,12.403871,17.192903
1-1871,15.098065,13.398387,13.080000,6.364194,11.342258,7.759677,10.297419,8.350000,8.979355,8.666774,14.392581,16.360000
1-1872,14.383871,11.848387,13.732258,5.925161,10.823226,7.291290,11.287097,6.733548,9.109355,8.728387,12.170645,17.598710
1-1873,13.177742,12.202903,10.860968,5.454516,9.327419,7.419677,10.738710,7.880000,8.651290,7.596774,12.076452,14.684516
1-1874,20.990968,17.623548,17.514839,10.549355,15.858387,11.298387,14.112258,12.786452,14.691935,15.092903,19.929677,24.573226
1-1875,17.100323,15.812903,13.942581,8.081935,13.934194,8.437742,13.179677,12.601935,13.000645,11.064839,19.266774,21.713226


### Step 16. Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week (assume that the first week starts on January 2 1961) for the first 52 weeks.

In [80]:
first52 = data[1:].head(7 * 52).loc[:, "RPT":"MAL"]

first52['weekofyear'] = first52.index.map(lambda x: x.weekofyear)
grouped = first52.groupby('weekofyear')

def location_wide(fn):
    return grouped.agg(fn).apply(fn, axis = 1)

weekly_stats = pd.DataFrame(list(map(location_wide, [np.min, np.max, np.mean, np.std]))).transpose()
weekly_stats.columns = ["min", "max", "mean", "std"]
weekly_stats


,min,max,mean,std
weekofyear,,,,
1,0.00,18.50,9.632143,1.249558
2,0.50,20.71,8.353214,0.745440
3,0.00,20.79,9.261667,0.862366
4,0.00,27.71,16.721429,1.241432
5,3.63,24.21,11.800357,1.038405
6,8.08,26.38,15.891548,0.988810
7,0.00,29.63,13.606786,1.270125
8,2.21,25.80,12.604286,0.760060
9,0.00,23.45,11.345357,1.411957
